# Burden

## Imports and Definitions

In [ ]:
import os
import torch
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt

from SCMP import SCMP
import DataGeneration as data
from CommonFunctions import *
from SocialMeasures import Burden

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)

PATH = "./Results/burden"
if not os.path.exists(PATH):
    os.makedirs(PATH)

## Data Loading

In [ ]:
# credit dataset
X, Y = data.load_credit_default_data()
X, Y = X[:3000], Y[:3000]
X, Y, Xval, Yval, Xtest, Ytest = data.split_validation_test(X, Y)

## Training

In [ ]:
x_dim = len(X[0])
scale = 1 # 1 / x_dim
epochs = 10
batch_size = 64

lambda_range = np.logspace(start=-2, stop=0.3, num=50)
print(lambda_range)

In [ ]:
results = {
    "lambdas": [],
    "accuracies": [],
    "burdens": []
}

print(f"---------- Training with lambda={0} ----------")
model_name = f"burden_{0:.3f}"
burden = Burden(0, x_dim, SCMP.linear_score, quad_cost_cvxpy_not_batched, quad_cost_torch, cost_const_kwargs={"scale": scale})  # Create burden from outside since reg=0.
benchmark_model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, 
                       social_measure_dict={"burden": burden}, strategic=True)
benchmark_model.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 5e-2}, epochs=epochs,
                    verbose="batches", path=PATH, model_name=model_name)

benchmark_model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, 
                       social_measure_dict={"burden": burden}, strategic=True)
benchmark_model.load_model(PATH, model_name)

results["lambdas"].append(0)
accuracy = benchmark_model.evaluate(Xtest, Ytest, strategic_data=True)
results["accuracies"].append(accuracy)
burden = benchmark_model.social_measure_dict["burden"].calc_burden(Xtest, Ytest, benchmark_model.w, benchmark_model.b, requires_grad=False).item()
results["burdens"].append(burden)
pd.DataFrame(results).to_csv(f"{PATH}/results.csv")

for lamb in lambda_range:
    print(f"---------- Training with lambda={lamb} ----------")
    model_name = f"burden_{lamb:.3f}"
    model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, burden_reg=lamb, strategic=True)
    model.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 5e-2},
                        epochs=epochs, verbose="batches", path=PATH, model_name=model_name)
    
    model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, burden_reg=lamb, strategic=True)
    model.load_model(PATH, model_name)
    
    results["lambdas"].append(lamb)
    accuracy = model.evaluate(Xtest, Ytest, strategic_data=True)
    results["accuracies"].append(accuracy)
    burden = model.social_measure_dict["burden"].calc_burden(Xtest, Ytest, model.w, model.b, requires_grad=False).item()
    results["burdens"].append(burden)
    pd.DataFrame(results).to_csv(f"{PATH}/results.csv")

## Show Results

In [ ]:
results = pd.read_csv("Results/burden/results.csv")
accuracies = results["accuracies"].to_list()
burdens = results["burdens"].to_list()
benchmark = accuracies[0]
# accuracies, burdens = accuracies[1:], burdens[1:]

In [ ]:
fig, ax = plt.subplots(1, figsize=(24, 6))

_ = ax.set_xlabel("social burden (decreasing)")
_ = ax.set_xlim([2.4, 0])
_ = ax.set_ylabel("accuracy")
_ = ax.set_title("Social burden tradeoff")
_ = ax.axhline(y=benchmark, linewidth=3, linestyle="--", color="gray", label="Benchmark")
_ = ax.plot(burdens, accuracies, 'o',  label="SERM")
_ = ax.legend()